In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

theta = 'pi/4'

folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

In [ ]:
Purify.get_tstamps_and_offsets(contains = 'Purify', verbose = False)

In [ ]:
thetas =['pi/4','pi/5','pi/6','pi/8']

num_first_clicks = np.zeros(len(thetas))
total_syncs = np.zeros(len(thetas))
entanglement_time = np.zeros(len(thetas))
reset_time = np.zeros(len(thetas))
store_time = np.zeros(len(thetas))
msmt_time = np.zeros(len(thetas))

for i, theta in enumerate(thetas):
    
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()
    Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify.get_tstamps_and_offsets(contains = 'Purify', verbose = False)
    num_first_clicks[i], total_syncs[i], entanglement_time[i], reset_time[i], store_time[i], msmt_time[i] = Purify.calculate_sequence_time(max_w2 = 500,print_details = True, return_click_prob = True)

In [ ]:
realpopvals = np.array([0.49214634,  0.39799117,  0.27649945,  0.1760004])
total_time =entanglement_time + reset_time + store_time + msmt_time
plt.plot(realpopvals,entanglement_time/total_time, label = 'Ent. time')
plt.plot(realpopvals,reset_time/total_time, label = 'Reset time')
plt.plot(realpopvals,store_time/total_time, label = 'Store time')
plt.plot(realpopvals,msmt_time/total_time, label = 'Measurement time')
plt.legend(bbox_to_anchor=(1.5, 1.025), loc='upper right', ncol=1)

print entanglement_time/total_time

In [ ]:
realpopvals = np.array([0.49214634,  0.39799117,  0.27649945,  0.1760004])

probs = num_first_clicks/total_syncs

fit_result = fit.fit1d(realpopvals, probs,common.fit_line,0,1,fixed = [0])
plot.plot_fit1d(fit_result)

In [ ]:
b = fit_result['params_dict']['b']
error = fit_result['error_dict']['b']
print b, error

## Rate of generating two raw states

In [ ]:
#### calculates the generation rate of two raw states. Does not take overhead into account (LDE elements only --> 7 us).
reload(pa)
thetas =['pi/4','pi/5','pi/6','pi/8']

num_two_raw_states = np.zeros(len(thetas))
total_syncs = np.zeros(len(thetas))

for i, theta in enumerate(thetas):
    
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()
    Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify.get_tstamps_and_offsets(contains = 'Purify', verbose = False)
    Purify.load_raw_data()
    Purify.correct_pq_times()
    
    Purify.apply_temporal_filters_to_prefiltered_data(verbose=False)
    num_two_raw_states[i] += Purify.apply_sync_filter_w1_w2(verbose = True,max_w2 = 50, return_events = True)
    
    for j in range(len(Purify.lt4_dict['/PQ_sync_number-1'])):
        total_syncs[i] += Purify.lt4_dict['/PQ_sync_number-1'][j][-1]

In [ ]:
## rate generating two raw states for each superposition angle in Hz
num_two_raw_states/(total_syncs*7e-6)
